In [1]:
!pip install tensorflow==2.2

     |████████████████████████████████| 516.2 MB 3.1 kB/s 
     |████████████████████████████████| 3.0 MB 48.0 MB/s 
     |████████████████████████████████| 2.9 MB 51.1 MB/s 
     |████████████████████████████████| 454 kB 60.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

In [ ]:
!pip install deepctr[cpu]

     |████████████████████████████████| 147 kB 14.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import math
import gc
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import DeepFM, xDeepFM, nfm, difm, wdl, pnn, FNN
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
valid_data = pd.read_csv('/content/drive/MyDrive/Taobao/valid_data_10_percent.csv')

In [ ]:
num_train = len(valid_data[valid_data['userid']<=valid_data.loc[int(0.8*len(valid_data))]['userid']])
num_train

1758557

In [ ]:
# numerics = ['price']  

numerics = ['price', 
       'num_show_last_hour_user_adgroup', 'num_click_last_hour_user_adgroup',
       'num_show_last_day_user_adgroup', 'num_click_last_day_user_adgroup',
       'num_show_last_hour_user_cate', 'num_click_last_hour_user_cate',
       'num_show_last_day_user_cate', 'num_click_last_day_user_cate',
       'num_show_last_hour_user_customer', 'num_click_last_hour_user_customer',
       'num_show_last_day_user_customer', 'num_click_last_day_user_customer',
       'num_show_last_hour_user_brand', 'num_click_last_hour_user_brand',
       'num_show_last_day_user_brand', 'num_click_last_day_user_brand',
       'num_show_last_hour_user_all', 'num_click_last_hour_user_all',
       'num_show_last_day_user_all', 'num_click_last_day_user_all',
       'num_show_last_hour_all_adgroup', 'num_click_last_hour_all_adgroup',
       'num_show_last_hour_all_cate', 'num_click_last_hour_all_cate',
       'num_show_last_hour_all_customer', 'num_click_last_hour_all_customer',
       'num_show_last_hour_all_brand', 'num_click_last_hour_all_brand'
       ]

categories = ['cate_id','customer', 'brand', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
              'pvalue_level', 'shopping_level', 'occupation', 'new_user_class_level','hour','weekday']

target = 'clk'
valid_cols = numerics + categories + [target]

In [ ]:
valid_data = valid_data[valid_cols]

for col in numerics:
  valid_data[col].fillna(0, inplace=True)
  # valid_data[col] = pd.qcut(valid_data[col], 20, labels=False, duplicates = "drop")

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in categories:
  print(feat)
  lbe = LabelEncoder()
  valid_data[feat] = lbe.fit_transform(valid_data[feat].astype(str))
  
mms = MinMaxScaler(feature_range=(0, 1))
valid_data[numerics] = mms.fit_transform(valid_data[numerics])

cate_id
customer
brand
cms_segid
cms_group_id
final_gender_code
age_level
pvalue_level
shopping_level
occupation
new_user_class_level
hour
weekday


In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=valid_data[feat].nunique(), embedding_dim=2)
              for i,feat in enumerate(categories)] + [DenseFeat(feat, 1,)
              for feat in numerics]

# dnn_feature_columns = fixlen_feature_columns
# linear_feature_columns = fixlen_feature_columns

linear_feature_columns = [SparseFeat(feat, vocabulary_size=valid_data[feat].nunique(), embedding_dim=2)
              for i,feat in enumerate(categories)]

dnn_feature_columns = [DenseFeat(feat, 1,)
              for feat in numerics]

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(len(fixlen_feature_columns))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
42


In [ ]:
# 3.generate input data for model
tmp = valid_data.loc[:int(num_train)]
train, val = train_test_split(tmp, test_size=0.2, random_state=123)

In [ ]:
X_train = train[feature_names]
y_train = train[target].values
X_val = val[feature_names]
y_val = val[target].values

In [ ]:
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0, sampling_strategy=0.2)
# cc = ClusterCentroids(random_state=0)
# smo = SMOTENC(categorical_features = list(range(0,len(categories))), random_state=0, sampling_strategy=0.2,k_neighbors=3,n_jobs=-1)
X_train_sample, y_train_sample = rus.fit_resample(X_train, y_train)

In [ ]:
# X_train_input = {name:X_train[name] for name in feature_names}
X_train_input = {name:X_train_sample[name] for name in feature_names}
X_val_input = {name:X_val[name] for name in feature_names}

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,RMSprop,Adadelta

In [ ]:
from tensorflow.keras import backend as K

def focal_loss(alpha=0.25, gamma=2.0):

    def focal_loss_fixed(y_true, y_pred):
      
        ones = K.ones_like(y_true)
        alpha_t = y_true*alpha + (ones-y_true)*(1-alpha)

        p_t = y_true*y_pred + (ones-y_true)*(ones-y_pred) + K.epsilon()

        focal_loss = -alpha_t * K.pow((ones-p_t),gamma) * K.log(p_t)

        return focal_loss

    return focal_loss_fixed

In [ ]:
import tensorflow as tf
tf.random.set_seed(123)

# 4.Define Model, train, predict and evaluate
# model = wdl.WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64), l2_reg_dnn=1e-5, dnn_dropout=0, task='binary', seed=123)
model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64), task='binary', dnn_dropout=0, l2_reg_dnn=0, seed=123)
# model = pnn.PNN(dnn_feature_columns, dnn_hidden_units=(256, 128, 64), l2_reg_embedding=1e-05, l2_reg_dnn=1e-05, seed=123, dnn_dropout=0.1, task='binary')

# model.compile(RMSprop(0.005), loss="binary_crossentropy", metrics=["AUC"],)
model.compile(RMSprop(0.005), loss=[focal_loss(gamma=2.0, alpha=0.25)], metrics=["AUC"],)

es = EarlyStopping(monitor='val_loss', patience=10)

# history = model.fit(X_train_input, y_train,
#            batch_size=8192, epochs=80, verbose=2, validation_data=(X_val_input,y_val), callbacks=[es])

history = model.fit(X_train_input, y_train_sample,
           batch_size=8192, epochs=80, verbose=2, validation_data=(X_val_input,y_val), callbacks=[es])

Epoch 1/80


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


53/53 - 7s - loss: 0.0461 - auc: 0.5390 - val_loss: 0.0301 - val_auc: 0.5976
Epoch 2/80
53/53 - 7s - loss: 0.0432 - auc: 0.5852 - val_loss: 0.0235 - val_auc: 0.6028
Epoch 3/80
53/53 - 5s - loss: 0.0428 - auc: 0.6090 - val_loss: 0.0237 - val_auc: 0.6093
Epoch 4/80
53/53 - 5s - loss: 0.0428 - auc: 0.6138 - val_loss: 0.0217 - val_auc: 0.6084
Epoch 5/80
53/53 - 5s - loss: 0.0427 - auc: 0.6202 - val_loss: 0.0216 - val_auc: 0.6126
Epoch 6/80
53/53 - 5s - loss: 0.0426 - auc: 0.6222 - val_loss: 0.0261 - val_auc: 0.6116
Epoch 7/80
53/53 - 6s - loss: 0.0426 - auc: 0.6242 - val_loss: 0.0245 - val_auc: 0.6107
Epoch 8/80
53/53 - 5s - loss: 0.0425 - auc: 0.6266 - val_loss: 0.0239 - val_auc: 0.6122
Epoch 9/80
53/53 - 5s - loss: 0.0425 - auc: 0.6269 - val_loss: 0.0235 - val_auc: 0.6091
Epoch 10/80
53/53 - 5s - loss: 0.0425 - auc: 0.6287 - val_loss: 0.0252 - val_auc: 0.6115
Epoch 11/80
53/53 - 5s - loss: 0.0425 - auc: 0.6270 - val_loss: 0.0247 - val_auc: 0.6126
Epoch 12/80
53/53 - 5s - loss: 0.0425 - a

In [ ]:
test = valid_data.loc[int(num_train):]
X_test_input = {name:test[name] for name in feature_names}
y_test = test[target].values

In [ ]:
pred_ans = model.predict(X_test_input, batch_size=8192)
print("test AUC", round(roc_auc_score(y_test, pred_ans), 4))
print("test Logloss", round(log_loss(y_test, pred_ans), 4))

test AUC 0.6123
test Logloss 0.374


In [ ]:
# pred_ans_2 = model.predict(X_test_input, batch_size=8192)
# print("test AUC", round(roc_auc_score(y_test, pred_ans_2), 4))
# print("test Logloss", round(log_loss(y_test, pred_ans_2), 4))

test AUC 0.5995
test Logloss 0.1987


In [ ]:
pd.DataFrame(pred_ans).to_csv('/content/drive/MyDrive/Taobao/deepfm_focal_rus_20.csv',index=False,header=None)